# Input Text Clean-up

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cp -av "/content/drive/MyDrive/NLP Project/Dataset.7z" /content/

'/content/drive/MyDrive/NLP Project/Dataset.7z' -> '/content/Dataset.7z'


In [ ]:
!pip install pyunpack
!pip install patool
#%mkdir snapdata
from pyunpack import Archive
Archive('/content/Dataset.7z').extractall('/content/')

     |████████████████████████████████| 77 kB 6.2 MB/s 


In [ ]:
import pandas as pd
import os
import numpy as np
#import plotly.express as px
import random
import nltk
from nltk.corpus import stopwords

SEED = 54
random.seed(SEED)

In [ ]:
import nltk
from nltk.corpus import stopwords        
from nltk.stem import PorterStemmer      
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
nltk.download('punkt')      # Punkt Sentence Tokenizer (https://www.kite.com/python/docs/nltk.punkt)
nltk.download('stopwords')  # Stopwords (https://gist.github.com/sebleier/554280) 
nltk.download('wordnet')    # Wordnet NLTk (https://www.nltk.org/howto/wordnet.html)
!nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
/bin/bash: -c: line 0: syntax error near unexpected token `'omw-1.4''
/bin/bash: -c: line 0: `nltk.download('omw-1.4')'


In [ ]:
!pip install fuzzywuzzy
import re

from fuzzywuzzy import process as fuzzy_proc
master_entities=[]
master_entities_text=[]
#code to load all standard entities for fuzzy matching & correction
with open('/content/MasterEntities.csv', 'r') as rf:
  for i in rf:
    master_entities.append(i.strip())
#master_entities=master_entities[1:] #skipped header

#code to load all standard entities for fuzzy matching & correction
with open('/content/MasterEntities_Text.csv', 'r') as rf1:
  for i in rf1:
    master_entities_text.append(i.strip())


def entity_correction(row, tgt_type):
  corrected_entity=[]
  for i in row[tgt_type]:
    match_score=fuzzy_proc.extractOne(i, master_entities)
    if match_score[1]>90:
      corrected_entity.append(match_score[0])
    else:
      corrected_entity.append(i)
  return corrected_entity

def entity_clean(entity_list):
  corrected_entity_list=[]
  for text in entity_list:
    text = re.sub('[^a-zA-Z0-9 ]', ' ', text)
    text = str.lower(text)
    #print(text)
    #lz = WordNetLemmatizer()
    #text.lemmatize(text) 
    corrected_entity_list.append(text)
  return corrected_entity_list

def text_correction(text_list):
  corrected_text_list=[]
  for i in text_list:
    match_score=fuzzy_proc.extractOne(i, master_entities_text)
    if match_score[1]>90:
      corrected_text_list.append(match_score[0])
    else:
      corrected_text_list.append(i)
  return corrected_text_list

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
import pandas as pd
from collections import OrderedDict
import re
import copy

# Prepare target variable
def clean_text(text):
    #text = text.replace('\r', ' ').replace('\n', ' ')
    text = re.sub('[^a-zA-Z0-9 ]', ' ', text)
    text = str.lower(text)
    text=word_tokenize(text)
    #print(text)
    #stop_words = set(stopwords.words('english'))
    #text = [t for t in text if not t in stop_words]
    #print(text)
    lz = WordNetLemmatizer()
    text = [lz.lemmatize(t) for t in text]
    before_list = copy.deepcopy(text)
    text = text_correction(text)
    after_list = copy.deepcopy(text)
    diff_set = set(after_list)-set(before_list)

    if len(diff_set)>0:
      print("Diff",diff_set)
      #print("Before",before_list)
      #print("After",after_list)

    text = ' '.join(text)
    #print(text)
    text = text.title()

    #print(text)
    return text

# Prepare target variable
def prepare_target(classval, classtype):
    target = ""
    if len(classval) > 0:
        target = f"[B{classtype}] " + " [SEP] ".join(classval) + f" [E{classtype}]"

    return target

def prepare_entiry(classval,classtype):
    target = ""
    if len(classval) > 0:
      if (classtype=='H'):
        target = ['B-HERO']*len(classval)
        #target = [0] * len(classval)
      if (classtype=='VL'):
        target = ['B-VIL']*len(classval)
        #target = [1] * len(classval)
      if (classtype=='VC'):
        target = ['B-VIC']*len(classval)
        #target = [2] * len(classval)
      if (classtype=='O'):
        target = ['B-OTH']*len(classval)
        #target = [3] * len(classval)
    else:
      target=[]

    return target

def mergeclasses(class_data: list):
    #random.shuffle(class_data)
    target = ""
    for _class in class_data:
        target+=_class + " "
    
    return target.strip()

def uniques(xs):
    return list(OrderedDict().fromkeys(xi for x in xs for xi in x))

def get_dataset(filename):
    covid_path = "Dataset/covid/annotations"
    politics_path = "Dataset/politics/annotations"

    covid_anno = pd.read_json(f"./{covid_path}/{filename}.jsonl", lines = True).reset_index(drop=True)
    politics_anno = pd.read_json(f"./{politics_path}/{filename}.jsonl", lines = True).reset_index(drop=True)

    covid_entities = covid_anno.apply(lambda x: x.hero + x.villain + x.victim + x.other, axis=1)
    politics_entities = politics_anno.apply(lambda x: x.hero + x.villain + x.victim + x.other, axis=1)


    final_ds = pd.concat([covid_anno, politics_anno], axis =0, ignore_index=True)

    #final_ds = final_ds[0:10]
    #print(final_ds)

    #final_ds['OCR_cleaned'] = (final_ds['OCR'].pipe(hero.clean))
    #final_ds['OCR_cleaned'] = final_ds['OCR']
    final_ds['OCR_cleaned'] = final_ds.apply(lambda x: clean_text(x.OCR), axis=1)
    
    final_ds["_Hero_Entity"] = final_ds.apply(lambda x: prepare_entiry(x.hero,"H"), axis=1)
    final_ds["_Villain_Entity"] = final_ds.apply(lambda x: prepare_entiry(x.villain,"VL"), axis=1) 
    final_ds["_Victim_Entity"] = final_ds.apply(lambda x: prepare_entiry(x.victim,"VC"), axis=1)
    final_ds["_Other_Entity"] = final_ds.apply(lambda x: prepare_entiry(x.other,"O"), axis=1)

    final_ds['villain_c']=final_ds.apply(lambda row: entity_clean(row.villain), axis=1)
    final_ds['victim_c']=final_ds.apply(lambda row: entity_clean(row.victim), axis=1)
    final_ds['hero_c']=final_ds.apply(lambda row: entity_clean(row.hero), axis=1)
    final_ds['other_c']=final_ds.apply(lambda row: entity_clean(row.other), axis=1)

    final_ds['villain_c1']=final_ds.apply(lambda row: entity_correction(row, 'villain_c'), axis=1)
    final_ds['victim_c1']=final_ds.apply(lambda row: entity_correction(row, 'victim_c'), axis=1)
    final_ds['hero_c1']=final_ds.apply(lambda row: entity_correction(row, 'hero_c'), axis=1)
    final_ds['other_c1']=final_ds.apply(lambda row: entity_correction(row, 'other_c'), axis=1)

    #--df['E']=(df.B+df.C).map(set).map(list)
    final_ds["Target"] = (final_ds.hero+final_ds.villain+final_ds.victim+final_ds.other).map(list)
    final_ds["Target_c"] = (final_ds.hero_c1+final_ds.villain_c1+final_ds.victim_c1+final_ds.other_c1).map(list)
    #print(set(final_ds.Target_c) - set(final_ds.Target))

    final_ds["Target_Entity"] = (final_ds._Hero_Entity+final_ds._Villain_Entity+final_ds._Victim_Entity+final_ds._Other_Entity).map(list)
    
    #--final_ds["Target_Entity"] = final_ds[['_Hero_Entity', '_Villain_Entity', '_Victim_Entity', '_Other_Entity']].apply(uniques, axis=1)
    #--final_ds["Target"] = final_ds[['hero', 'villain', 'victim', 'other']].apply(uniques, axis=1)
    #--final_ds["Target_Entity"] = final_ds[['_Hero_Entity', '_Villain_Entity', '_Victim_Entity', '_Other_Entity']].apply(uniques, axis=1)
    '''
    #final_ds["Target"] = final_ds.apply(lambda x: mergeclasses([x["hero"],x["villain"],x["victim"],x["other"]]), axis = 1)
    #final_ds["Target_Entity"] = final_ds.apply(lambda x: mergeclasses([x["_Hero_Entity"],x["_Villain_Entity"],x["_Victim_Entity"],x["_Other_Entity"]]), axis = 1)
    
    # = final_ds.apply(lambda x: prepare_target(x.hero, x.villain, x.victim, x.other), axis=1)
    final_ds["_Hero"] = final_ds.apply(lambda x: prepare_target(x.hero, "H"), axis=1)
    final_ds["_Villain"] = final_ds.apply(lambda x: prepare_target(x.villain, "VIL"), axis=1)
    final_ds["_Victim"] = final_ds.apply(lambda x: prepare_target(x.victim, "VIC"), axis=1)
    final_ds["_Other"] = final_ds.apply(lambda x: prepare_target(x.other, "OTH"), axis=1)
    final_ds["Target"] = final_ds.apply(lambda x: mergeclasses([x["hero"],x["villain"],x["victim"],x["other"]]), axis = 1)
    final_ds["All_Entities"] = final_ds.apply(lambda x: x.hero + x.villain + x.victim + x.other, axis=1)
    #final_ds["Target"] = final_ds["Target"].apply(lambda x: x.strip())
    '''
    # Remove column name 'A'
    final_ds = final_ds.drop(['hero','villain','victim','other','_Hero_Entity','_Villain_Entity','_Victim_Entity','_Other_Entity'], axis = 1)
    return final_ds

In [ ]:
text = "hello ?>< how are u..i am memes fantastic?"
clean_text(text)

'Hello How Are U I Am Meme Fantastic'

In [ ]:
!pwd

/content


In [ ]:
train = get_dataset("train")
valid = get_dataset("val")

Diff {'sanders'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'vladimir'}
Diff {'american'}
Diff {'sanders'}
Diff {'american'}
Diff {'coronavirus'}
Diff {'coronavirus'}
Diff {'american'}
Diff {'coronavirus'}
Diff {'trump'}
Diff {'coronavirus'}
Diff {'virus'}
Diff {'barack', 'sanders'}
Diff {'coronavirus'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'grand'}
Diff {'american'}
Diff {'american'}
Diff {'covid'}
Diff {'american'}
Diff {'coronavirus'}
Diff {'coronavirus'}
Diff {'vaccine'}
Diff {'american'}
Diff {'coronavirus'}
Diff {'coronavirus'}
Diff {'covid'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'coronavirus'}
Diff {'american'}
Diff {'vaccine'}
Diff {'american'}
Diff {'virus'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'trump'}
Diff {'coronavirus'}
Diff {'coronavirus'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'a

Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'coronavirus'}
Diff {'coronavirus'}
Diff {'american'}
Diff {'harris'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'trump'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'libertarian'}
Diff {'sanders'}
Diff {'republican', 'harris'}
Diff {'republican'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'sanders'}
Diff {'american'}
Diff {'american'}
Diff {'barack'}
Diff {'american'}
Diff {'republican'}
Diff {'american'}
Diff {'american'}
Diff {'american'}
Diff {'american'}


In [ ]:
train
train[train.apply(lambda row: all(i in row.Target_c for i in row.Target), axis=1)==False]

,OCR,image,OCR_cleaned,villain_c,victim_c,hero_c,other_c,villain_c1,victim_c1,hero_c1,other_c1,Target,Target_c,Target_Entity
36,This is Hob. He voted for Trump. This is Bob's...,covid_memes_600.png,This Is Hob He Voted For Trump This Is Bob S F...,[],[],[],"[donald trump, bill clinton, hilary clinton, h...",[],[],[],"[donald trump, bill clinton, hillary clinton, ...","[donald trump, bill clinton, hilary clinton, h...","[donald trump, bill clinton, hillary clinton, ...","[B-OTH, B-OTH, B-OTH, B-OTH]"
40,"re Photos\nIF YOU LOVE AMERICA,\nYOU'RE GENERA...",covid_memes_649.png,Re Photo If You Love American You Re Generally...,[liberal],[],[conservative],"[charlie kirk, america, usa]",[liberal],[],[conservative],"[charlie kirk, american, usa]","[conservative, liberal, charlie kirk, america,...","[conservative, liberal, charlie kirk, american...","[B-HERO, B-VIL, B-OTH, B-OTH, B-OTH]"
43,JUST HAD THE COVID-19\nVACCINE\nFEELING GREAT!...,covid_memes_46.png,Just Had The Covid 19 Vaccine Feeling Great Ma...,[covid19 vaccine],[],[],"[covid 19 vaccine, covid vaccine]",[covid vaccine],[],[],"[covid vaccine, covid vaccine]","[covid19 vaccine, covid 19 vaccine, covid vacc...","[covid vaccine, covid vaccine, covid vaccine]","[B-VIL, B-OTH, B-OTH]"
46,Me when I get my covid vaccine\nMemeZila.com\n,covid_memes_47.png,Me When I Get My Covid Vaccine Memezila Com,[],[],[],"[covid19 vaccine, covid vaccine]",[],[],[],"[covid vaccine, covid vaccine]","[covid19 vaccine, covid vaccine]","[covid vaccine, covid vaccine]","[B-OTH, B-OTH]"
47,HECHEVS\nTHE WHITE HC\nWASHINGTON\nNIK NEWS SP...,covid_memes_722.png,Hechevs The White Hc Washington Nik News Speci...,[],[],[],"[donald trump, dr anthony fauci, white house,...",[],[],[],"[donald trump, dr anthony fauci, white house,...","[donald trump, dr. anthony fauci, white house,...","[donald trump, dr anthony fauci, white house,...","[B-OTH, B-OTH, B-OTH, B-OTH]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5522,LET US BIULD THE ENTERPRISE\nWEAREALL\nOFUCKED\n,memes_957.png,Let U Biuld The Enterprise Weareall Ofucked,[],[],[],"[kim jong un, barack obama, kim jong un]",[],[],[],"[kim jong un, barack obama, kim jong un]","[kim jong un, barack obama, kim jong-un]","[kim jong un, barack obama, kim jong un]","[B-OTH, B-OTH, B-OTH]"
5526,HITLER BLAMED JEWSFOR\nTHE WOES OF\nGERMANY\nR...,memes_5036.png,Hitler Blamed Jewsfor The Woe Of Germany Remem...,"[donald trump, adolf hitler]","[jews, mexicans]",[],"[germany, america, mexican]","[donald trump, adolf hitler]","[jews, mexicans]",[],"[germany, american, mexican]","[donald trump, adolf hitler, jews, mexicans, g...","[donald trump, adolf hitler, jews, mexicans, g...","[B-VIL, B-VIL, B-VIC, B-VIC, B-OTH, B-OTH, B-OTH]"
5538,"Because of me, the Republican party\nlost to a...",memes_4950.png,Because Of Me The Republican Party Lost To A B...,[],[black people],[],"[americans, george bush, george walker bush, r...",[],[black people],[],"[american, george bush, george walker bush, re...","[black people, americans, george bush, george ...","[black people, american, george bush, george w...","[B-VIC, B-OTH, B-OTH, B-OTH, B-OTH, B-OTH]"
5545,J.B. PRITZER'S NET\nWORTH: $3.5 BILLION\nTHERE...,memes_3413.png,J B Pritzer S Net Worth 3 5 Billion There S No...,[j b pritzer],[],[],"[socialist, j b pritzer, libertarians]",[j b pritzer],[],[],"[socialist, j b pritzer, libertarian]","[j.b. pritzer, socialist, j. b. pritzer, liber...","[j b pritzer, socialist, j b pritzer, liber...","[B-VIL, B-OTH, B-OTH, B-OTH]"


In [ ]:
train.head()

,OCR,image,OCR_cleaned,villain_c,victim_c,hero_c,other_c,villain_c1,victim_c1,hero_c1,other_c1,Target,Target_c,Target_Entity
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]",[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]"
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[],[],[],[uk government],[uk government],[B-VIL]
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],[],[],[thais],[hong kong],"[thais, hong kong]","[thais, hong kong]","[B-HERO, B-OTH]"
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]",[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]"
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]",[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]"


In [ ]:
valid.head()

,OCR,image,OCR_cleaned,villain_c,victim_c,hero_c,other_c,villain_c1,victim_c1,hero_c1,other_c1,Target,Target_c,Target_Entity
0,Herman Caino\nOTHEHermancain\nMasks will not b...,covid_memes_2457.png,Herman Caino Othehermancain Mask Will Not Be M...,[herman cain],[],[],"[donald trump, covid, coronavirus, mask, ameri...",[herman cain],[],[],"[donald trump, covid, coronavirus, mask, ameri...","[herman cain, donald trump, covid, coronavirus...","[herman cain, donald trump, covid, coronavirus...","[B-VIL, B-OTH, B-OTH, B-OTH, B-OTH, B-OTH]"
1,Let's go to China\nto save them from\nI'm the ...,covid_memes_257.png,Let S Go To China To Save Them From I M The Ma...,[],[],[],"[batman, china, coronavirus]",[],[],[],"[batman, china, coronavirus]","[batman, china, coronavirus]","[batman, china, coronavirus]","[B-OTH, B-OTH, B-OTH]"
2,WHO WOULD WIN?\nThanos\nOne hantavirus boy\nSo...,covid_memes_280.png,Who Would Win Thanos One Hantavirus Boy Sorry ...,[],[],[],"[hantavirus, thanos, hanta virus, thanos vs ha...",[],[],[],"[hantavirus, thanos, hanta virus, thanos vs ha...","[hantavirus, thanos, hanta virus, thanos vs ha...","[hantavirus, thanos, hanta virus, thanos vs ha...","[B-OTH, B-OTH, B-OTH, B-OTH]"
3,Dotors: *Working 24x7 relentlessly to\nsave li...,covid_memes_459.png,Dotors Working 24X7 Relentlessly To Save Life ...,"[goan aunties, goan uncles]",[doctors],[goicho saib],"[goencho saib, goan people, coronavirus, goa, ...","[goan aunties, goan uncles]",[doctors],[goicho saib],"[goencho saib, goan people, coronavirus, goa, ...","[goicho saib, goan aunties, goan uncles, docto...","[goicho saib, goan aunties, goan uncles, docto...","[B-HERO, B-VIL, B-VIL, B-VIC, B-OTH, B-OTH, B-..."
4,Occupy Danocrats\na at 12\nDREAKNG: Barack Cba...,covid_memes_622.png,Occupy Danocrats A At 12 Dreakng Barack Cbaria...,[],[],[joe biden],[barack obama],[],[],[joe biden],[barack obama],"[joe biden, barack obama]","[joe biden, barack obama]","[B-HERO, B-OTH]"


In [ ]:
train.to_parquet('train_dataframe_1.parquet')
valid.to_parquet('valid_dataframe_1.parquet')

In [ ]:
train = train.drop(['villain_c','victim_c','hero_c','other_c','Target'], axis = 1)
valid = valid.drop(['villain_c','victim_c','hero_c','other_c','Target'], axis = 1)
train.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]"
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL]
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]"
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]"
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]"


In [ ]:
valid.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity
0,Herman Caino\nOTHEHermancain\nMasks will not b...,covid_memes_2457.png,Herman Caino Othehermancain Mask Will Not Be M...,[herman cain],[],[],"[donald trump, covid, coronavirus, mask, ameri...","[herman cain, donald trump, covid, coronavirus...","[B-VIL, B-OTH, B-OTH, B-OTH, B-OTH, B-OTH]"
1,Let's go to China\nto save them from\nI'm the ...,covid_memes_257.png,Let S Go To China To Save Them From I M The Ma...,[],[],[],"[batman, china, coronavirus]","[batman, china, coronavirus]","[B-OTH, B-OTH, B-OTH]"
2,WHO WOULD WIN?\nThanos\nOne hantavirus boy\nSo...,covid_memes_280.png,Who Would Win Thanos One Hantavirus Boy Sorry ...,[],[],[],"[hantavirus, thanos, hanta virus, thanos vs ha...","[hantavirus, thanos, hanta virus, thanos vs ha...","[B-OTH, B-OTH, B-OTH, B-OTH]"
3,Dotors: *Working 24x7 relentlessly to\nsave li...,covid_memes_459.png,Dotors Working 24X7 Relentlessly To Save Life ...,"[goan aunties, goan uncles]",[doctors],[goicho saib],"[goencho saib, goan people, coronavirus, goa, ...","[goicho saib, goan aunties, goan uncles, docto...","[B-HERO, B-VIL, B-VIL, B-VIC, B-OTH, B-OTH, B-..."
4,Occupy Danocrats\na at 12\nDREAKNG: Barack Cba...,covid_memes_622.png,Occupy Danocrats A At 12 Dreakng Barack Cbaria...,[],[],[joe biden],[barack obama],"[joe biden, barack obama]","[B-HERO, B-OTH]"


In [ ]:
train.to_parquet('train_dataframe_2.parquet')
valid.to_parquet('valid_dataframe_2.parquet')

# BERT

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 880 kB 73.7 MB/s 
     |████████████████████████████████| 6.6 MB 49.3 MB/s 
     |████████████████████████████████| 596 kB 60.7 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a7cc064819c7a5c29aa6b692c8746fda53a69aa098897254221c9ddf3ac9c47f
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")

nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy='first')


In [ ]:
example = train["OCR_cleaned"][3]
ner_results = nlp(example)
print(example)
print(ner_results)

So I Order This Foce Mask To Protect Ogainst Fhe Corond Virus Hooold Up It Say Something Here Made In China
[{'entity_group': 'PER', 'score': 0.48739478, 'word': 'Ogainst', 'start': 37, 'end': 44}, {'entity_group': 'ORG', 'score': 0.41888413, 'word': 'Corond', 'start': 49, 'end': 55}, {'entity_group': 'LOC', 'score': 0.99795663, 'word': 'China', 'start': 102, 'end': 107}]


In [ ]:
train.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity,BERT_Target,BERT_Target_Entity
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]","[PER, PER, MISC]","[Bernie, Elizabeth, Dankest]"
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL],[MISC],[Brexit]
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]","[ORG, MISC, LOC, MISC]","[Kwai Gkwa, Thai, Hong Kong, Thai]"
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]","[PER, ORG, LOC]","[Ogainst, Corond, China]"
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]","[LOC, PER, PER, PER, PER]","[Ja, Joe Biden, Kamala Harris, Bernie Sanders,..."


In [ ]:
train = pd.read_parquet('/content/train_dataframe_2.parquet', engine='pyarrow')
valid = pd.read_parquet('/content/valid_dataframe_2.parquet', engine='pyarrow')
train.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]"
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL]
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]"
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]"
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]"


In [ ]:
#train = train.drop(['BERT_Target','BERT_Target_Entity'], axis = 1)
train.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]"
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL]
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]"
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]"
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]"


In [ ]:
%cp -av  "/content/train_dataframe_1.parquet" "/content/drive/MyDrive/NLP Project/"
%cp -av  "/content/train_dataframe_2.parquet" "/content/drive/MyDrive/NLP Project/"
%cp -av  "/content/valid_dataframe_1.parquet" "/content/drive/MyDrive/NLP Project/"
%cp -av  "/content/valid_dataframe_2.parquet" "/content/drive/MyDrive/NLP Project/"

'/content/train_dataframe_1.parquet' -> '/content/drive/MyDrive/NLP Project/train_dataframe_1.parquet'
'/content/train_dataframe_2.parquet' -> '/content/drive/MyDrive/NLP Project/train_dataframe_2.parquet'
'/content/valid_dataframe_1.parquet' -> '/content/drive/MyDrive/NLP Project/valid_dataframe_1.parquet'
'/content/valid_dataframe_2.parquet' -> '/content/drive/MyDrive/NLP Project/valid_dataframe_2.parquet'


In [ ]:
#train = train[0:10]
#string = string.replace('\r', '').replace('\n', '')

train["BERT_Target"] = ""
train["BERT_Target_Entity"] = ""

for i in range(train.shape[0]):
    Bert_Entities = []
    Bert_Entity_Names = []

    text=str(train['OCR_cleaned'][i])
    if len(text)>0:
      out = nlp(text)
      if len(out)>0:
        for x in out:
          Bert_Entities.append(x['entity_group'])
          Bert_Entity_Names.append(x['word'])
    train["BERT_Target"][i] = Bert_Entities
    train["BERT_Target_Entity"][i] = Bert_Entity_Names

train

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity,BERT_Target,BERT_Target_Entity
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]","[PER, PER, MISC]","[Bernie, Elizabeth, Dankest]"
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL],[MISC],[Brexit]
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]","[ORG, MISC, LOC, MISC]","[Kwai Gkwa, Thai, Hong Kong, Thai]"
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]","[PER, ORG, LOC]","[Ogainst, Corond, China]"
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]","[LOC, PER, PER, PER, PER]","[Ja, Joe Biden, Kamala Harris, Bernie Sanders,..."
...,...,...,...,...,...,...,...,...,...,...,...
5547,Trump could shoot someone\non the Senate floor...,memes_5039.png,Trump Could Shoot Someone On The Senate Floor ...,[donald trump],[],[],"[senate floor, republican]","[donald trump, senate floor, republican]","[B-VIL, B-OTH, B-OTH]","[PER, ORG, LOC, MISC]","[Trump, Senate, Floor, Republican]"
5548,MANY PEOPLE ASK\nME WHY ALL MY SCHOOL\nRECORDS...,memes_2635.png,Many People Ask Me Why All My School Record Ar...,[],[],[],"[school, university, joe biden]","[school, university, joe biden]","[B-OTH, B-OTH, B-OTH]",[],[]
5549,my bes\nfriend\nmy\nmother\nconsclence\nmy the...,memes_1384.png,My Be Friend My Mother Consclence My Therapist...,[],[],[],"[msnbc, bernie sanders, democratic party, joe ...","[msnbc, bernie sanders, democratic party, joe ...","[B-OTH, B-OTH, B-OTH, B-OTH, B-OTH]","[PER, MISC]","[Ciny B, Democratic]"
5550,THE\nN-WORD\nPASS\nSigned and\napproved by\nBe...,memes_944.png,The N Word Pas Signed And Approved By Beak Obana,[],[],[],[barack obama],[barack obama],[B-OTH],"[ORG, PER]","[Word, Beak Obana]"


In [ ]:
train.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity,BERT_Target,BERT_Target_Entity
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]","[PER, PER, MISC]","[Bernie, Elizabeth, Dankest]"
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL],[MISC],[Brexit]
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]","[ORG, MISC, LOC, MISC]","[Kwai Gkwa, Thai, Hong Kong, Thai]"
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]","[PER, ORG, LOC]","[Ogainst, Corond, China]"
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]","[LOC, PER, PER, PER, PER]","[Ja, Joe Biden, Kamala Harris, Bernie Sanders,..."


In [ ]:
train.to_parquet('train_dataframe_3.parquet')

In [ ]:
%cp -av  "/content/train_dataframe_3.parquet" "/content/drive/MyDrive/NLP Project/"

'/content/train_dataframe_3.parquet' -> '/content/drive/MyDrive/NLP Project/train_dataframe_3.parquet'


In [ ]:
train = pd.read_parquet('/content/train_dataframe_3.parquet', engine='pyarrow')
train.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity,BERT_Target,BERT_Target_Entity
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]","[PER, PER, MISC]","[Bernie, Elizabeth, Dankest]"
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL],[MISC],[Brexit]
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]","[ORG, MISC, LOC, MISC]","[Kwai Gkwa, Thai, Hong Kong, Thai]"
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]","[PER, ORG, LOC]","[Ogainst, Corond, China]"
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]","[LOC, PER, PER, PER, PER]","[Ja, Joe Biden, Kamala Harris, Bernie Sanders,..."


In [ ]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz
import re

x=train
x["total_entity_count"] = 0
x["text_entity_match_count"] = 0
total_row_matches = 0

for i in range(0,x.shape[0]):
#for i in range(0,2):
  match_count = 0
  inp = x["OCR_cleaned"][i]
  inp = re.sub('[^a-zA-Z0-9 ]', ' ', str(inp))
  inp = str.lower(inp)
  
  a = inp.split()
  b = x["Target_c"][i]
  #print(a,b,len(a),len(b))
  in_count = len(b)
  #print(in_count,match_count)
  #print(a,b,in_count)
  print()
  
  
  #print(a,b)
  for c in b:
    for d in a:
      match_score=fuzz.ratio(str.lower(c).strip(), str.lower(d).strip())
      if match_score >=50:
        #print(c,d,match_score)
        match_count+=1
        break
  #print(in_count,match_count)
  x["total_entity_count"][i] = in_count
  x["text_entity_match_count"][i] = match_count

  if match_count>0:
    total_row_matches+=1
  
print(total_row_matches)

'''
    match_score=fuzzy_proc.extractOne(i, master_entities)
    if match_score[1]>90:
      corrected_entity.append(match_score[0])
    else:
      corrected_entity.append(i)
  return corrected_entity
'''


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Streaming output truncated to the last 5000 lines.























































































































































































































































































































































































































































































































































































































































































































































































































































































































































































'\n    match_score=fuzzy_proc.extractOne(i, master_entities)\n    if match_score[1]>90:\n      corrected_entity.append(match_score[0])\n    else:\n      corrected_entity.append(i)\n  return corrected_entity\n'

In [ ]:
x.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity,BERT_Target,BERT_Target_Entity,total_entity_count,text_entity_match_count
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]","[PER, PER, MISC]","[Bernie, Elizabeth, Dankest]",2,2
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL],[MISC],[Brexit],1,0
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]","[ORG, MISC, LOC, MISC]","[Kwai Gkwa, Thai, Hong Kong, Thai]",2,2
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]","[PER, ORG, LOC]","[Ogainst, Corond, China]",4,4
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]","[LOC, PER, PER, PER, PER]","[Ja, Joe Biden, Kamala Harris, Bernie Sanders,...",4,4


In [ ]:
total = x["total_entity_count"].sum()
matched = x["text_entity_match_count"].sum()
print("%match",float(matched/total))

%match 0.8172890259221195


In [ ]:
x = x.drop(['in_count','match_count'], axis = 1)
x.to_parquet('train_dataframe_4.parquet') 

In [ ]:
x = pd.read_parquet('/content/train_dataframe_4.parquet', engine='pyarrow')
x.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity,BERT_Target,BERT_Target_Entity,total_entity_count,text_entity_match_count
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]","[PER, PER, MISC]","[Bernie, Elizabeth, Dankest]",2,2
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL],[MISC],[Brexit],1,0
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]","[ORG, MISC, LOC, MISC]","[Kwai Gkwa, Thai, Hong Kong, Thai]",2,2
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]","[PER, ORG, LOC]","[Ogainst, Corond, China]",4,4
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]","[LOC, PER, PER, PER, PER]","[Ja, Joe Biden, Kamala Harris, Bernie Sanders,...",4,4


In [ ]:
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz


x["in_count"] = 0
x["match_count"] = 0

total_row_matches = 0
for i in range(0,x.shape[0]):
  a = x["Target_c"][i]
  b = x["BERT_Target_Entity"][i]
  in_count = len(a)
  x["in_count"][i] = in_count
  match_count = 0
  #print(a,b)
  for c in a:
    for d in b:
      match_score=fuzz.ratio(str.lower(c).strip(), str.lower(d).strip())
      if match_score >=50:
        #print(c,d)
        match_count+=1
        break
    x["match_count"][i] = match_count
  if match_count>0:
    total_row_matches+=1
  
print(total_row_matches)

'''
    match_score=fuzzy_proc.extractOne(i, master_entities)
    if match_score[1]>90:
      corrected_entity.append(match_score[0])
    else:
      corrected_entity.append(i)
  return corrected_entity
'''

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


3744


'\n    match_score=fuzzy_proc.extractOne(i, master_entities)\n    if match_score[1]>90:\n      corrected_entity.append(match_score[0])\n    else:\n      corrected_entity.append(i)\n  return corrected_entity\n'

In [ ]:
x.head()
total = x["in_count"].sum()
matched = x["match_count"].sum()
print("%match",float(matched/total))

%match 0.48041566746602715


In [ ]:
#!pip install fuzzywuzzy
from fuzzywuzzy import fuzz


x["in_count_rev"] = 0
x["match_count_rev"] = 0

total_row_matches = 0
for i in range(0,x.shape[0]):
  b = x["Target_c"][i]
  a = x["BERT_Target_Entity"][i]
  in_count = len(a)
  x["in_count_rev"][i] = in_count
  match_count = 0
  #print(a,b)
  for c in a:
    for d in b:
      match_score=fuzz.ratio(str.lower(c).strip(), str.lower(d).strip())
      if match_score >=50:
        #print(c,d)
        match_count+=1
        break
    x["match_count_rev"][i] = match_count
  if match_count>0:
    total_row_matches+=1
  
print(total_row_matches)

'''
    match_score=fuzzy_proc.extractOne(i, master_entities)
    if match_score[1]>90:
      corrected_entity.append(match_score[0])
    else:
      corrected_entity.append(i)
  return corrected_entity
'''
x.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


3744


,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity,BERT_Target,BERT_Target_Entity,total_entity_count,text_entity_match_count,in_count,match_count,in_count_rev,match_count_rev
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]","[PER, PER, MISC]","[Bernie, Elizabeth, Dankest]",2,2,2,2,3,2
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL],[MISC],[Brexit],1,0,1,0,1,0
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]","[ORG, MISC, LOC, MISC]","[Kwai Gkwa, Thai, Hong Kong, Thai]",2,2,2,2,4,3
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]","[PER, ORG, LOC]","[Ogainst, Corond, China]",4,4,4,3,3,2
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]","[LOC, PER, PER, PER, PER]","[Ja, Joe Biden, Kamala Harris, Bernie Sanders,...",4,4,4,4,5,4


In [ ]:
x.head()
total = x["in_count_rev"].sum()
matched = x["match_count_rev"].sum()
print("%match",float(matched/total))

%match 0.6564110697010547


In [ ]:
x.to_parquet('train_dataframe_5.parquet') 

In [ ]:
x.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity,BERT_Target,BERT_Target_Entity,total_entity_count,text_entity_match_count,in_count,match_count,in_count_rev,match_count_rev
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]","[PER, PER, MISC]","[Bernie, Elizabeth, Dankest]",2,2,2,2,3,2
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL],[MISC],[Brexit],1,0,1,0,1,0
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]","[ORG, MISC, LOC, MISC]","[Kwai Gkwa, Thai, Hong Kong, Thai]",2,2,2,2,4,3
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]","[PER, ORG, LOC]","[Ogainst, Corond, China]",4,4,4,3,3,2
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]","[LOC, PER, PER, PER, PER]","[Ja, Joe Biden, Kamala Harris, Bernie Sanders,...",4,4,4,4,5,4


In [ ]:
%cp -av  "/content/train_dataframe_5.parquet" "/content/drive/MyDrive/NLP Project/"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
train = pd.read_parquet('/content/drive/MyDrive/NLP Project/train_dataframe_5.parquet', engine='pyarrow')
train.head()

Mounted at /content/drive


,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity,BERT_Target,BERT_Target_Entity,total_entity_count,text_entity_match_count,in_count,match_count,in_count_rev,match_count_rev
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]","[PER, PER, MISC]","[Bernie, Elizabeth, Dankest]",2,2,2,2,3,2
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL],[MISC],[Brexit],1,0,1,0,1,0
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]","[ORG, MISC, LOC, MISC]","[Kwai Gkwa, Thai, Hong Kong, Thai]",2,2,2,2,4,3
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]","[PER, ORG, LOC]","[Ogainst, Corond, China]",4,4,4,3,3,2
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]","[LOC, PER, PER, PER, PER]","[Ja, Joe Biden, Kamala Harris, Bernie Sanders,...",4,4,4,4,5,4


In [ ]:
train["OCR_cleaned"][3]

'Kwai Gkwa 0964 Nnevvy Applause To Thai From Hong Kong Why Thai Did Not Get Hurt Cultural Difference Just Stay In The Wall'

In [ ]:
total = train["total_entity_count"].sum()
matched = train["text_entity_match_count"].sum()
print("%match",matched/total)
print("%match",matched,total)

%match 0.8172890259221195
%match 14314 17514


In [ ]:
in_total = train["in_count"].sum()
in_matched = train["match_count"].sum()
print("%match",in_matched/in_total)
print("%match",in_matched/matched)
print("%match",in_matched,matched,in_total)

%match 0.48041566746602715
%match 0.5878161240743328
%match 8414 14314 17514


In [ ]:
in_total_rev = train["in_count_rev"].sum()
in_matched_rev = train["match_count_rev"].sum()

print("%match",in_matched_rev,in_total_rev,total)

%match 8278 12611 17514


In [ ]:
train["Net"] = train["in_count_rev"] - train["in_count"]

In [ ]:
train.head()

,OCR,image,OCR_cleaned,villain_c1,victim_c1,hero_c1,other_c1,Target_c,Target_Entity,BERT_Target,BERT_Target_Entity,total_entity_count,text_entity_match_count,in_count,match_count,in_count_rev,match_count_rev,Net
0,Bernie or Elizabeth?\nBe informed.Compare them...,covid_memes_18.png,Bernie Or Elizabeth Be Informed Compare Them O...,[],[],[],"[bernie sanders, elizabeth warren]","[bernie sanders, elizabeth warren]","[B-OTH, B-OTH]","[PER, PER, MISC]","[Bernie, Elizabeth, Dankest]",2,2,2,2,3,2,1
1,Extending the\nBrexit deadline until\nOctober ...,covid_memes_19.png,Extending The Brexit Deadline Until October 31...,[uk government],[],[],[],[uk government],[B-VIL],[MISC],[Brexit],1,0,1,0,1,0,0
2,kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...,covid_memes_252.png,Kwai Gkwa 0964 Nnevvy Applause To Thai From Ho...,[],[],[thais],[hong kong],"[thais, hong kong]","[B-HERO, B-OTH]","[ORG, MISC, LOC, MISC]","[Kwai Gkwa, Thai, Hong Kong, Thai]",2,2,2,2,4,3,2
3,"So, I order this\nfoce mask to\nprotect ogains...",covid_memes_255.png,So I Order This Foce Mask To Protect Ogainst F...,[china],[],[],"[face mask, made in china, coronavirus]","[china, face mask, made in china, coronavirus]","[B-VIL, B-OTH, B-OTH, B-OTH]","[PER, ORG, LOC]","[Ogainst, Corond, China]",4,4,4,3,3,2,-1
4,best candidate for\nJA\n2020\njoe biden\nKamal...,covid_memes_20.png,Best Candidate For Ja 2020 Joe Biden Kamala Ha...,[],[],[joe biden],"[bernie sanders, kamala harris, tiktok]","[joe biden, bernie sanders, kamala harris, tik...","[B-HERO, B-OTH, B-OTH, B-OTH]","[LOC, PER, PER, PER, PER]","[Ja, Joe Biden, Kamala Harris, Bernie Sanders,...",4,4,4,4,5,4,1


In [ ]:
train[train["Net"]>0]
print(sum(train[train["Net"]>0]["Net"]))

2503


In [ ]:
#Total as input - 17514
#Text Entities - 14314 
#Matched Entities - 8414
#Total matches - 12611 
# Non text matched entities - 2503